# LangGraph Basics: Building Your First Agent

## Introduction

LangGraph is a library for building stateful, multi-actor applications with LLMs. It extends LangChain with the ability to create cyclical graphs, which are essential for agent-like behaviors.

### What You'll Learn
- Core LangGraph concepts: Nodes, Edges, State
- How to build a simple graph
- How to execute and visualize graphs
- State management fundamentals

### Prerequisites
```bash
pip install langgraph langchain langchain-openai
```

## Core Concepts

### 1. StateGraph - The Foundation

A `StateGraph` is the main building block in LangGraph. It defines:
- **State Schema**: What information flows through the graph
- **Nodes**: Functions that process the state
- **Edges**: Connections between nodes

```python
from langgraph.graph import StateGraph
from typing import TypedDict

# Define state schema
class AgentState(TypedDict):
    messages: list
    current_step: str
```

### 2. Nodes - Processing Units

Nodes are functions that:
- Take the current state as input
- Perform some operation
- Return updates to the state

```python
def my_node(state: AgentState) -> AgentState:
    # Process state
    state["messages"].append("Processed!")
    return state
```

### 3. Edges - Flow Control

Edges connect nodes and control flow:
- **Regular edges**: Always go from A → B
- **Conditional edges**: Choose next node based on state

```python
# Regular edge
graph.add_edge("node_a", "node_b")

# Conditional edge
graph.add_conditional_edges(
    "node_a",
    lambda state: "node_b" if state["condition"] else "node_c"
)
```

## Example 1: Simple Linear Graph

Let's build the simplest possible graph: A → B → C

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

# 1. Define the state
class SimpleState(TypedDict):
    messages: list
    count: int

# 2. Define nodes
def node_a(state: SimpleState) -> SimpleState:
    print("Executing Node A")
    state["messages"].append("A")
    state["count"] += 1
    return state

def node_b(state: SimpleState) -> SimpleState:
    print("Executing Node B")
    state["messages"].append("B")
    state["count"] += 1
    return state

def node_c(state: SimpleState) -> SimpleState:
    print("Executing Node C")
    state["messages"].append("C")
    state["count"] += 1
    return state

# 3. Build the graph
workflow = StateGraph(SimpleState)

# Add nodes
workflow.add_node("a", node_a)
workflow.add_node("b", node_b)
workflow.add_node("c", node_c)

# Add edges
workflow.set_entry_point("a")  # Start at node A
workflow.add_edge("a", "b")    # A → B
workflow.add_edge("b", "c")    # B → C
workflow.add_edge("c", END)    # C → END

# 4. Compile the graph
app = workflow.compile()

# 5. Run the graph
initial_state = {"messages": [], "count": 0}
result = app.invoke(initial_state)

print("\nFinal State:")
print(f"Messages: {result['messages']}")
print(f"Count: {result['count']}")

### Explanation

1. **State Definition**: We define what data flows through the graph
2. **Node Functions**: Each node modifies the state
3. **Graph Construction**: We connect nodes with edges
4. **Compilation**: Convert the graph definition to an executable
5. **Execution**: Run the graph with initial state

```
Initial: {messages: [], count: 0}
   ↓
Node A: {messages: ["A"], count: 1}
   ↓
Node B: {messages: ["A", "B"], count: 2}
   ↓
Node C: {messages: ["A", "B", "C"], count: 3}
```

## Example 2: Conditional Routing

Now let's add decision-making: the graph chooses different paths based on state.

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Literal

# 1. Define state
class ConditionalState(TypedDict):
    value: int
    path: str
    steps: list

# 2. Define nodes
def start_node(state: ConditionalState) -> ConditionalState:
    print(f"Starting with value: {state['value']}")
    state["steps"].append("start")
    return state

def even_node(state: ConditionalState) -> ConditionalState:
    print("Taking EVEN path")
    state["path"] = "even"
    state["steps"].append("even")
    state["value"] = state["value"] // 2
    return state

def odd_node(state: ConditionalState) -> ConditionalState:
    print("Taking ODD path")
    state["path"] = "odd"
    state["steps"].append("odd")
    state["value"] = state["value"] * 3 + 1
    return state

# 3. Define routing function
def route_based_on_value(state: ConditionalState) -> Literal["even", "odd"]:
    """Route to 'even' or 'odd' node based on current value"""
    if state["value"] % 2 == 0:
        return "even"
    else:
        return "odd"

# 4. Build graph
workflow = StateGraph(ConditionalState)

workflow.add_node("start", start_node)
workflow.add_node("even", even_node)
workflow.add_node("odd", odd_node)

# Set entry point
workflow.set_entry_point("start")

# Add conditional edge from start
workflow.add_conditional_edges(
    "start",
    route_based_on_value,
    {"even": "even", "odd": "odd"}
)

# Both paths lead to END
workflow.add_edge("even", END)
workflow.add_edge("odd", END)

# 5. Compile and run
app = workflow.compile()

# Test with even number
print("=== Test 1: Even Number ===")
result1 = app.invoke({"value": 10, "path": "", "steps": []})
print(f"Result: {result1}\n")

# Test with odd number
print("=== Test 2: Odd Number ===")
result2 = app.invoke({"value": 7, "path": "", "steps": []})
print(f"Result: {result2}")

### Conditional Edge Visualization

```
         ┌─────────┐
         │  START  │
         └────┬────┘
              │
         ┌────▼────┐
         │  Check  │
         │  Value  │
         └────┬────┘
              │
       ┌──────┴──────┐
       │             │
    (even)        (odd)
       │             │
       ▼             ▼
  ┌────────┐    ┌────────┐
  │  Even  │    │  Odd   │
  │  Node  │    │  Node  │
  └────┬───┘    └───┬────┘
       │            │
       └─────┬──────┘
             │
             ▼
        ┌─────────┐
        │   END   │
        └─────────┘
```

## Example 3: Loops and Cycles

One of LangGraph's most powerful features: creating loops for iterative processing.

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Literal

# 1. Define state
class LoopState(TypedDict):
    count: int
    max_iterations: int
    history: list

# 2. Define nodes
def process_node(state: LoopState) -> LoopState:
    """Process and increment counter"""
    state["count"] += 1
    state["history"].append(f"Iteration {state['count']}")
    print(f"Processing iteration {state['count']}...")
    return state

# 3. Define routing function
def should_continue(state: LoopState) -> Literal["continue", "end"]:
    """Decide whether to continue looping or end"""
    if state["count"] < state["max_iterations"]:
        return "continue"
    else:
        return "end"

# 4. Build graph with loop
workflow = StateGraph(LoopState)

workflow.add_node("process", process_node)

workflow.set_entry_point("process")

# Add conditional edge that can loop back
workflow.add_conditional_edges(
    "process",
    should_continue,
    {
        "continue": "process",  # Loop back to process
        "end": END              # Or end
    }
)

# 5. Compile and run
app = workflow.compile()

result = app.invoke({
    "count": 0,
    "max_iterations": 5,
    "history": []
})

print("\nFinal Result:")
print(f"Total iterations: {result['count']}")
print(f"History: {result['history']}")

### Loop Visualization

```
    ┌─────────┐
    │  START  │
    └────┬────┘
         │
    ┌────▼────┐
    │ Process │ ◄────┐
    └────┬────┘      │
         │           │
    ┌────▼────┐      │
    │  Check  │      │
    │  Count  │      │
    └────┬────┘      │
         │           │
      [count < max?] │
         │           │
    Yes──┴───────────┘
         │
    No   │
         ▼
    ┌─────────┐
    │   END   │
    └─────────┘
```

## Example 4: Real Agent with LLM

Now let's build a simple agent that uses an actual LLM!

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
import operator

# 1. Define state with message accumulation
class AgentState(TypedDict):
    messages: Annotated[list, operator.add]  # Messages will accumulate
    current_step: str

# 2. Initialize LLM
llm = ChatOpenAI(temperature=0, model="gpt-4")

# 3. Define nodes
def call_model(state: AgentState) -> AgentState:
    """Call the LLM with current messages"""
    messages = state["messages"]
    response = llm.invoke(messages)
    return {
        "messages": [response],
        "current_step": "model_called"
    }

def should_continue(state: AgentState) -> str:
    """Determine if we should continue or end"""
    last_message = state["messages"][-1]
    
    # If the AI says 'DONE', we end
    if "DONE" in last_message.content:
        return "end"
    else:
        return "continue"

# 4. Build graph
workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)

workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "agent",
        "end": END
    }
)

# 5. Compile
app = workflow.compile()

# 6. Run
print("Simple LLM Agent Example")
print("="*50)

initial_messages = [
    HumanMessage(content="""Count from 1 to 3, one number at a time. 
    After each number, I'll say 'next'. When you reach 3, say 'DONE'.""")
]

result = app.invoke({
    "messages": initial_messages,
    "current_step": "start"
})

print("\nConversation:")
for msg in result["messages"]:
    role = "Human" if isinstance(msg, HumanMessage) else "AI"
    print(f"{role}: {msg.content}")

### Key Concepts in This Example

#### 1. Annotated State
```python
messages: Annotated[list, operator.add]
```
- The `Annotated` type tells LangGraph HOW to update the state
- `operator.add` means: append new messages to existing ones
- Without this, new messages would replace old ones

#### 2. Message Types
- `HumanMessage`: User input
- `AIMessage`: LLM response
- `SystemMessage`: System instructions

#### 3. Conditional Flow
- The agent can loop back to itself
- It decides when to stop based on output
- This is the foundation of agentic behavior!

## State Update Strategies

LangGraph offers different ways to update state:

### 1. Replace (Default)
```python
class State(TypedDict):
    value: int  # New value replaces old value
```

### 2. Append (using operator.add)
```python
class State(TypedDict):
    messages: Annotated[list, operator.add]  # Append to list
```

### 3. Custom Reducer
```python
def merge_dicts(existing, new):
    return {**existing, **new}

class State(TypedDict):
    data: Annotated[dict, merge_dicts]  # Custom merge
```

## Debugging and Visualization

### 1. Print State at Each Step

In [ ]:
def debug_node(state: dict) -> dict:
    print(f"Current state: {state}")
    return state

# Add debug nodes between processing nodes
workflow.add_node("debug", debug_node)
workflow.add_edge("process", "debug")
workflow.add_edge("debug", "next_process")

### 2. Stream Intermediate Results

In [ ]:
# Stream each step's output
for step in app.stream({"messages": [HumanMessage("Hello")]}):
    print(f"Step output: {step}")
    print("-" * 50)

### 3. Get Graph Structure

In [ ]:
# Get the graph as Mermaid diagram
from IPython.display import Image, display

try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception:
    # Fallback to text representation
    print(app.get_graph().draw_ascii())

## Common Patterns

### Pattern 1: Retry Logic

In [ ]:
class RetryState(TypedDict):
    attempts: int
    max_attempts: int
    success: bool

def try_action(state: RetryState) -> RetryState:
    state["attempts"] += 1
    # Simulate action that might fail
    import random
    state["success"] = random.random() > 0.5
    return state

def should_retry(state: RetryState) -> str:
    if state["success"]:
        return "success"
    elif state["attempts"] < state["max_attempts"]:
        return "retry"
    else:
        return "failed"

workflow = StateGraph(RetryState)
workflow.add_node("action", try_action)
workflow.set_entry_point("action")
workflow.add_conditional_edges(
    "action",
    should_retry,
    {
        "retry": "action",
        "success": END,
        "failed": END
    }
)

### Pattern 2: Multi-Step Processing

In [ ]:
# Data pipeline: Load → Clean → Transform → Save

class PipelineState(TypedDict):
    raw_data: str
    cleaned_data: str
    transformed_data: str
    saved: bool

def load_data(state: PipelineState) -> PipelineState:
    state["raw_data"] = "raw,data,here"
    return state

def clean_data(state: PipelineState) -> PipelineState:
    state["cleaned_data"] = state["raw_data"].replace(",", " ")
    return state

def transform_data(state: PipelineState) -> PipelineState:
    state["transformed_data"] = state["cleaned_data"].upper()
    return state

def save_data(state: PipelineState) -> PipelineState:
    print(f"Saving: {state['transformed_data']}")
    state["saved"] = True
    return state

workflow = StateGraph(PipelineState)
workflow.add_node("load", load_data)
workflow.add_node("clean", clean_data)
workflow.add_node("transform", transform_data)
workflow.add_node("save", save_data)

workflow.set_entry_point("load")
workflow.add_edge("load", "clean")
workflow.add_edge("clean", "transform")
workflow.add_edge("transform", "save")
workflow.add_edge("save", END)

## Best Practices

### 1. State Design
- Keep state minimal - only what you need
- Use TypedDict for clear schemas
- Choose appropriate update strategies

### 2. Node Design
- Each node should do one thing well
- Keep nodes pure functions when possible
- Handle errors within nodes

### 3. Graph Design
- Start simple, add complexity gradually
- Use clear, descriptive node names
- Add safety limits (max iterations)

### 4. Testing
- Test nodes independently first
- Test routing logic separately
- Use streaming for debugging

### 5. Error Handling

In [ ]:
def safe_node(state: dict) -> dict:
    try:
        # Node logic here
        result = risky_operation()
        state["result"] = result
        state["error"] = None
    except Exception as e:
        state["error"] = str(e)
        state["result"] = None
    return state

## Exercises

### Exercise 1: Build a Calculator Graph
Create a graph that:
1. Takes two numbers and an operation (+, -, *, /)
2. Routes to the appropriate operation node
3. Returns the result

### Exercise 2: Add Validation
Extend the calculator to:
1. Validate inputs before calculation
2. Handle division by zero
3. Return error messages for invalid inputs

### Exercise 3: Build a Countdown
Create a graph that:
1. Counts down from a given number
2. Prints each number
3. Stops at zero

### Exercise 4: Multi-Path Processing
Create a graph that:
1. Classifies input text as "question" or "statement"
2. Routes to different processing nodes
3. Formats output appropriately

## Key Takeaways

- **StateGraph** is the foundation - defines state schema and flow
- **Nodes** are functions that transform state
- **Edges** control flow between nodes (regular or conditional)
- **Cycles** enable iterative processing and agent-like behavior
- **State updates** can use different strategies (replace, append, custom)
- **Streaming** and **debugging** tools help during development

## Next Steps

In the next notebook, we'll explore:
- Advanced state management techniques
- Integrating external tools
- Building real agents with memory
- Implementing common agent patterns

---

**Next**: [03_state_management_and_tools.ipynb](./03_state_management_and_tools.ipynb)